In [4]:
%matplotlib inline
from IPython.display import Image
from qiskit import *
from qiskit_ibm_runtime import *
from qiskit.quantum_info import Statevector
from qiskit.visualization import *
from qiskit_aer import *

# Cargar credenciales de IBM Quantum
service = QiskitRuntimeService()

### Aprendiendo sobre los efectos de la interferencia entre cúbits

Uno de los beneficios de la computación cuántica es su capacidad para entrelazar estos principios de tal manera que, al explicar uno, se puede describir fácilmente el otro. Hicimos esto anteriormente con respecto a la interferencia. Vamos a revisarlo y ver dónde hemos encontrado este fenómeno y su uso hasta ahora.

Primero, recuerda que, al inicio de este capítulo, describimos el experimento de la doble rendija. Allí, discutimos cómo un electrón puede actuar como una onda y como una partícula. Al actuar como una onda, vimos que el experimento ilustraba cómo los electrones viajaban y aterrizaban en ciertos puntos de la pantalla de observación. El patrón que mostraban era generalmente uno que reconocemos de la física clásica como interferencia de ondas.

El patrón tenía resultados probabilísticos a lo largo de la pantalla, como se muestra en la pantalla de observación. El centro de la pantalla tiene la mayor cantidad de electrones, y las áreas vacías a ambos lados tienen poco o ningún electrón. Esto se debe a la interferencia constructiva y destructiva de las ondas.

Hay dos tipos de interferencia, a saber:

1. **Constructiva**: La interferencia constructiva ocurre cuando los picos de dos ondas se suman y la amplitud resultante es igual a la suma positiva total de las dos ondas individuales.

2. **Destructiva**: La interferencia destructiva ocurre de manera similar a la interferencia constructiva, excepto que las amplitudes de las ondas son opuestas; al sumarlas, las ondas se cancelan entre sí.

El siguiente diagrama ilustra la interferencia constructiva y destructiva de las ondas cuando se combinan:

![image.png](./Class_01/Figures/interference.png)

Uno proviene del estado base $ |0\rangle $, mientras que el otro proviene del estado base $ |1\rangle $.  
**¿Recuerdas cuando partimos de uno de estos dos estados base de cúbits y preguntamos en qué lugar del eje X aterrizaría el qubit tras aplicar la puerta Hadamard?**  
Desde $ |0\rangle $, aterrizaría en el lado positivo del eje X, pero si colocáramos el cúbit en superposición partiendo del estado $ |1\rangle $, aterrizaría en el lado negativo del eje X.

Tener la capacidad de colocar el vector de estado del cúbit en el lado positivo o negativo del eje X nos permite ubicar el cúbit en un estado positivo o negativo. Muy similar a las ondas del diagrama anterior, que tienen amplitudes positivas (picos) y negativas (valles), los cúbits también pueden representar estados similares. Simplifiquemos esto reintroduciendo los dos valores en notación de Dirac: $ |+\rangle $ y $ |-\rangle $, donde:

- El estado $ |+\rangle $ representa el vector de estado en el lado positivo del eje X.
- El estado $ |-\rangle $ representa el vector de estado en el lado negativo del eje X.

Estas nuevas definiciones de vectores, que representan el estado vectorial de un cúbit en superposición, serán utilizadas por algunos algoritmos como una técnica para identificar ciertos valores y reaccionar a ellos utilizando interferencia. Esto incluye técnicas como **estimación de amplitud** o algoritmos de búsqueda como el **algoritmo de Grover**.


# Creando un circuito de teletransportación cuántica

En esta sección, crearemos un circuito de teletransportación cuántica para compartir el estado, $ |\psi\rangle $, de un qubit mediante la comunicación clásica de dos bits de información. ¿Por qué necesitaría compartir dos bits de información y no solo el estado del cúbit en sí? Bueno, la respuesta radica en el **teorema de no-clonación**.

Sin entrar en las pruebas de mecánica cuántica, el teorema establece que crear una copia de un cúbit a partir de un estado desconocido arbitrario no es posible, ya que no existe un operador unitario que pueda clonar todos los estados de un qubit en otro. Dicho esto, necesitamos buscar otros medios para transferir el estado de un qubit a otro. La **teletransportación cuántica** nos ayuda a lograrlo.

Para entender correctamente este ejemplo, echemos un vistazo al proceso general. Luego, podemos profundizar en los detalles y ver cómo podemos hacerlo posible. Como se mencionó anteriormente, el objetivo es tener un emisor —llamémosla Alice— que tiene un qubit en un estado arbitrario $ |\psi\rangle = \alpha|0\rangle + \beta|1\rangle $ y proporciona dos bits de información al receptor —llamémoslo Bob.

Alice luego enviará información a Bob de forma clásica (como comunicarse por llamada telefónica o mensaje de texto), quien realizará operaciones en un qubit que le permitirán generar el estado que Alice tenía. Decimos que **Alice tenía** ese estado porque, para enviar la información de los bits a Bob, Alice necesita realizar una medición en el estado arbitrario del qubit, lo que colapsará en valores binarios. Por lo tanto, se perderá toda la información cuántica del qubit.

Para ilustrar esto, vamos a crear un circuito que siga esta idea.

1. Ahora tenemos tres cúbits. El primer cúbit (`q[0]`) representará el estado desconocido $ |\psi\rangle $. El segundo y tercer cúbit (`q[1]` y `q[2]`) serán los cúbits entrelazados compartidos entre Alice y Bob, respectivamente.

In [6]:
# Crear el circuito de 3 qubits
qreg_q = QuantumRegister(3, 'q')
creg_c = ClassicalRegister(3, 'c')
qc = QuantumCircuit(qreg_q, creg_c)

2. Prepararemos el primer cúbit (`q[0]`) en un estado $ |\psi\rangle $ desconocido que ni Alice ni Bob sabrán. Por supuesto, nosotros lo conoceremos al crearlo. Sin embargo, la idea aquí es visualizar el estado desde el contexto de Alice y Bob. Para mantener este ejemplo simple, se aplicarán dos operadores unitarios: una puerta NOT y una puerta Z.

***Nota*** que se añadió una barrera después de establecer el estado $ |\psi\rangle $. Esto es simplemente para facilitar la visualización del circuito cuando lo dibujemos, para identificar a qué componente corresponde cada sección. En este caso, se trata del primer bloque que especifica $ |\psi\rangle $

In [7]:
qc.x(0)
qc.z(0)
qc.barrier()

CircuitInstruction(operation=Instruction(name='barrier', num_qubits=3, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(3, 'q'), 0), Qubit(QuantumRegister(3, 'q'), 1), Qubit(QuantumRegister(3, 'q'), 2)), clbits=())

3. Ahora que hemos preparado nuestro estado, que es conocido por nosotros pero desconocido tanto para Alice como para Bob, pasaremos al siguiente paso en nuestro diagrama de flujo: que Alice entrelace los otros dos cúbits (`q[1]` y `q[2]`) juntos. Alice conservará el qubit `q[1]` y enviará el qubit `q[2]` a Bob. Para entrelazar los dos qubits, aplicaremos una puerta Hadamard al qubit de Alice (`q[1]`), seguida de una puerta CNOT entre los dos qubits, donde el control está conectado al qubit de Alice y el objetivo está conectado al qubit de Bob. Incluiremos una barrera aquí, como hicimos anteriormente, para segmentar esta preparación.


In [8]:
qc.h(1)
qc.cx(1, 2)
qc.barrier()

CircuitInstruction(operation=Instruction(name='barrier', num_qubits=3, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(3, 'q'), 0), Qubit(QuantumRegister(3, 'q'), 1), Qubit(QuantumRegister(3, 'q'), 2)), clbits=())